In [ ]:
import time
import requests

api_key = "apk.c96bba2ab67b4394016c7499e0a66600862c1a59c1185ef709fc6c7c541beac2"
url = f"https://api-connect.eos.com/api/gdw/api?api_key={api_key}"

RATE_LIMIT = 10
MINUTE_IN_SECONDS = 60
payload = {
    "type": "mt_stats",
    "params":
        {
            "bm_type": ["NDVI", "MSI", "EVI"],
            "date_start": "2024-01-01",
            "date_end": "2024-01-30",
            "geometry":
                {
                    "coordinates":
                        [[
                            [77.48711792311, 28.5753746671189],
                            [77.487384802907, 28.5756620359534],
                            [77.487474656909, 28.575570172231],
                            [77.487215823739, 28.575301647044],
                            [77.48711792311, 28.5753746671189]
                        ]],
                    "type": "Polygon"
                },
            "reference": "ref_20200924-00-00",
            "max_cloud_cover_in_AOI": 10,
            "sensors": ["sentinel2"],
            "limit": 10
        }
}
headers = {
    'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, json=payload)
time.sleep(MINUTE_IN_SECONDS / RATE_LIMIT)
data = response.json()
task_id = data['task_id']
print(data)
task_status_url = f"https://api-connect.eos.com/api/gdw/api/{task_id}?api_key={api_key}"
task_response = requests.request("GET", task_status_url, headers=headers, json=payload)
task_data = task_response.json()
print(task_data)

#results = task_data['result']
#print(results)
#errors = task_data['errors']
#print(errors)

{'status': 'created', 'task_id': '786b029c-3cb6-467a-a1e6-feff8fd83dd6', 'req_id': 'a47a8169-63aa-4aaf-ac56-066145323a56', 'task_timeout': 345600}
{'client_id': 1607408, 'download_id': 1607408, 'token': 1607408, 'task_type': 'error', 'reference': 'ref_20200924-00-00', 'task_id': '786b029c-3cb6-467a-a1e6-feff8fd83dd6', 'type': 'mt_stats', 'hash_id': None, 'result_url': None, 'channel': None, 'req_id': None, 'result': None, 'success': None, 'png': None, 'error_message': {'error': "Invalid request in params: Additional properties are not allowed ('max_cloud_cover_in_AOI' was unexpected)"}}


In [ ]:
import requests
import json

url = "https://services.sentinel-hub.com/api/v1/process"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer <your-access-token>"  # Replace with your access token
}

payload = {
    "input": {
        "bounds": {
            "bbox": [13.822174072265625, 45.85080395917834, 13.828353881835938, 45.854201310382374],
            "properties": {
                "crs": "http://www.opengis.net/def/crs/EPSG/0/4326"
            }
        },
        "data": [
            {
                "type": "sentinel-2-l1c",
                "dataFilter": {
                    "timeRange": {
                        "from": "2023-06-01T00:00:00Z",
                        "to": "2023-06-30T23:59:59Z"
                    }
                }
            }
        ]
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [
            {
                "identifier": "default",
                "format": {
                    "type": "application/json"
                }
            }
        ]
    },
    "evalscript": "//VERSION=3\nfunction setup() {\n  return {\n    input: [\"B04\", \"B08\"],\n    output: { bands: 1 }\n  };\n}\n\nfunction evaluatePixel(sample) {\n  let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);\n  return [ndvi];\n}"
}

response = requests.post(url, headers=headers, data=json.dumps(payload))

if response.status_code == 200:
    data = response.json()
    print("NDVI Data:", data)
else:
    print("Error:", response.status_code, response.text)

<Response [422]>